In [3]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import keras
from sklearn.metrics import classification_report,confusion_matrix
import nltk
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv(r'/content/Train_and_validation.csv')
df

,Unnamed: 0,Unnamed: 0.1,id,text,bias
0,0,0,1,Besides these forests naturally developed fore...,0
1,1,1,2,Your mother cleans the rice by separating husk...,0
2,2,2,3,All free gifts of nature and all things which ...,0
3,3,3,4,Humanism - the concept that man is the measure...,0
4,4,4,5,Various designs in clothing are found dependin...,0
...,...,...,...,...,...
1147,1991,1991,1992,"Mulberry , soyabean , cotton , tobacco , flax ...",1
1148,1993,1993,1994,b. Self Employment : Picture : . To eradicate ...,1
1149,1995,1995,1996,Bring both the ends of threads together and pu...,1
1150,1997,1997,1998,PotassiumProtein synthesisEnables the opening ...,1


In [5]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer(language='english')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
sw = stopwords.words('english')
unwanted = {"he", "him", "his", "himself", "she", "she's", "her", "hers", "herself", "they", "them", "their", "theirs", "themselves"}
sw = [el for el in sw if el not in unwanted]
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'it', "it's", 'its', 'itself', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 'v

In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(sw)

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df.text = df.text.apply(clean_text)
df.text = df.text.str.replace('\d+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version.


In [8]:
import gensim
import gensim.downloader as gensim_api
nlp = gensim_api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
corpus = df.text

## create list of lists of unigrams
lst_corpus = []
for string in corpus:
   lst_words = string.split()
   lst_grams = [" ".join(lst_words[i:i+1]) 
               for i in range(0, len(lst_words), 1)]
   lst_corpus.append(lst_grams)

## detect bigrams and trigrams
bigrams_detector = gensim.models.phrases.Phrases(lst_corpus, 
                 delimiter=" ".encode(), min_count=5, threshold=10)
bigrams_detector = gensim.models.phrases.Phraser(bigrams_detector)
trigrams_detector = gensim.models.phrases.Phrases(bigrams_detector[lst_corpus], 
            delimiter=" ".encode(), min_count=5, threshold=10)
trigrams_detector = gensim.models.phrases.Phraser(trigrams_detector)

In [10]:
nlp = gensim.models.word2vec.Word2Vec(lst_corpus, size=300, window=8, min_count=1, sg=1, iter=30)

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

In [ ]:
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(lst_corpus)
dic_vocabulary = tokenizer.word_index
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X = kprocessing.sequence.pad_sequences(lst_text2seq, 
                    maxlen=15, padding="post", truncating="post")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, df.bias, test_size = 0.1, random_state = 42) 

In [ ]:
# lamba = -1
# for el in range(len(X_train)):
#   if X_train[el][-1] != 0:
#     lamba = max(lamba, len(X_train[el]))

# lamba

In [ ]:
## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dic_vocabulary)+1, 300))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

In [ ]:
## code attention layer
def attention_layer(inputs, neurons):
    x = layers.Permute((2,1))(inputs)
    x = layers.Dense(neurons, activation="softmax")(x)
    x = layers.Permute((2,1), name="attention")(x)
    x = layers.multiply([inputs, x])
    return x

## input
x_in = layers.Input(shape=(15,))
## embedding
x = layers.Embedding(input_dim=embeddings.shape[0],  
                     output_dim=embeddings.shape[1], 
                     weights=[embeddings],
                     input_length=15, trainable=False)(x_in)
## apply attention
x = attention_layer(x, neurons=15)
## 2 layers of bidirectional lstm
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2, 
                         return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(units=15, dropout=0.2))(x)
## final dense layers
x = layers.Dense(64, activation='relu')(x)
y_out = layers.Dense(1, activation='sigmoid')(x)
## compile
model = models.Model(x_in, y_out)
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
## encode y
dic_y_mapping = {n:label for n,label in 
                 enumerate(np.unique(Y_train))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
y_train = np.array([inverse_dic[y] for y in Y_train])
## train
training = model.fit(x=X_train, y=y_train, batch_size=32,
                     epochs=15, shuffle=True, verbose=1, 
                     validation_split=0.3)
# y_train.shape

In [ ]:
predicted_prob = model.evaluate(X_test, Y_test)
# predicted = [dic_y_mapping[np.argmax(pred)] for pred in
            #  predicted_prob]

In [ ]:
predicted_prob = model.predict(X_test)
y_pred = list(map(lambda x: 1 if x>0.5 else 0,predicted_prob))
print(classification_report(Y_test,y_pred))


In [ ]:
print(confusion_matrix(Y_test,y_pred))

In [ ]:
X,y = X,df.bias

In [ ]:
X

In [ ]:
predicted_prob = model.predict(X)
y_hat = list(map(lambda x: 1 if x>0.5 else 0,predicted_prob))
print(classification_report(Y_test,y_pred))


In [ ]:
df["pred"] = y_hat
df


In [ ]:
text_df = df.drop(['Unnamed: 0',	'Unnamed: 0.1'],axis=1).set_index('id')
text_df

In [ ]:
pair_df = pd.read_csv('/content/pairs.txt',names=['p1','p2','label'])
pair_df

In [ ]:
pred_label = []
for i in range(pair_df.shape[0]):
    pred_label.append(np.logical_xor(text_df.pred[pair_df.p1[i]],text_df.pred[pair_df.p2[i]]).astype(int))
#     break;

In [ ]:
pred_label

In [ ]:
# from sklearn.metrics import classification_report
print(classification_report(pair_df.label.values,pred_label))
print(confusion_matrix(pair_df.label.values,pred_label))
